In [ ]:
from collections import Counter
import itertools
from sklearn.neighbors import KDTree
import numpy as np


In [ ]:
def knn_predict(training_data, k, test_instance):
    """
    Prédit la classe d'une instance de test en fonction des k plus proches voisins.

    :param training_data: Ensemble de données d'entraînement avec des paires (features, label).
    :param k: Le nombre de voisins à considérer.
    :param test_instance: L'instance à classer.
    :return: La classe prédite pour l'instance de test.
    """
    # Extract features from training data
    features = [features for features, _ in training_data]

    # Build KDTree from training data features
    kdtree = KDTree(features)

    # Query KDTree to find k nearest neighbors
    _, indices = kdtree.query(test_instance, k=k)

    # Get labels of k nearest neighbors
    nearest_labels = [training_data[i][1] for i in indices]

    # Find the most common label among the neighbors
    most_common_label = Counter(nearest_labels).most_common(1)[0][0]

    return most_common_label



In [ ]:
def compute_inflSet(R, groups):
    """
    Calcule l'ensemble des points influencés par R.

    :param R: Ensemble des points à supprimer.
    :param groups: Les groupes actuels de points de données.
    :return: Un ensemble de points influencés par R.
    """
    # Pour simplifier, disons que tous les points sont influencés par R.
    # Dans une implémentation réelle, vous devrez vérifier si la suppression
    # de points dans R affecte les voisins les plus proches des points dans groups.
    return set(itertools.chain.from_iterable(groups)) - set(R)


In [ ]:
def ensemble(errSetGj_Ki, R, newSet_minus, newSet_plus):

    # Retirer les indices R et newSet_minus de errSetGj_Ki
    result = np.setdiff1d(errSetGj_Ki, R)
    result = np.setdiff1d(errSetGj_Ki, newSet_minus)

    # Ajouter les indices de newSet_plus à errSetGj_Ki
    result = np.union1d(errSetGj_Ki, newSet_plus)

    # Le tableau errSetGj_Ki est maintenant mis à jour
    print(result)


In [ ]:
def KNNLearnUpdate(R, Error, candidate_k_values):
    """
    Mise à jour incrémentielle du modèle KNN.

    :param R: Ensemble des éléments à supprimer de l'entraînement.
    :param Error: Contient des groupes et des ensembles d'erreurs.
    :param candidate_k_values: Les valeurs k candidates pour le KNN.
    :return: Le meilleur k basé sur la plus petite erreur.
    """
    # Récupération des groupes et des ensembles d'erreurs
    groups = []
    err_sets = []
    for elt in Error:
        groups.append(elt[0])
        err_sets.append(elt[1])

    # Calcul des nouveaux groupes en supprimant R
    new_groups = []
    for elt in groups:
        ensemble = [x for x in elt if not any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in R)]
        new_groups.append(ensemble)

    # Création du nouvel ensemble d'entraînement T'
    T_prime = set().union(*new_groups)

    # Initialisation de l'ensemble influencé par R
    inflSet = compute_inflSet(R, groups)  # Cette fonction doit être définie

    # Initialisation du dictionnaire pour les erreurs de chaque valeur k
    error_prime = []
    
    err_sets_prime = []
    
    errors_for_k = []

    # Itération sur chaque valeur k candidate
    for k in candidate_k_values:
        i = candidate_k_values.index(k)
        # Itération sur chaque groupe
        for j, Gj_prime in enumerate(new_groups):
            newSet_minus = []
            newSet_plus = []
            # Itération sur chaque élément de données dans le groupe influencé
            interGj_prime_inflSet = [x for x in Gj_prime if any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in inflSet)]
            for x, y in interGj_prime_inflSet:
                # Prédiction sur l'ensemble d'entraînement original
                data = [x for x in T if not any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in Gj)]
                data_prime = [x for x in T_prime if not any(np.array_equal(x[0], element[0]) and x[1] == element[1] for element in Gj_prime)]
                if knn_predict(data, k, x) == y and knn_predict(data_prime, k, x) != y:
                    newSet_plus.add((x, y))

                if knn_predict(data, k, x) != y and knn_predict(data_prime, k, x) == y:
                    newSet_minus.add((x, y))

            # Mise à jour de l'ensemble des erreurs pour le groupe j
            err_sets_prime[i][j] = ensemble(err_sets[j], R, newSet_minus, newSet_plus)
            error_prime[i][j] = len(err_sets[Gj_prime]) / len(Gj_prime)

        # Calcul de l'erreur moyenne pour la valeur k
        avg_error_k = sum([error_prime[i][j] for j in range(len(groups))]) / len(groups))
        errors_for_k[i] = avg_error_k

    # Trouver le meilleur k avec la plus petite erreur moyenne
    best_k = min(errors_for_k, key=errors_for_k.get)

    return best_k